Instalar librerías necesarias

In [ ]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importar librerías que se usarán

In [ ]:
import cv2
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
from numpy import asarray

Inicializar el detector de rostros

In [ ]:
detector = MTCNN()

Cargar el modelo personalizado entrenado previamente

In [ ]:
model = tf.keras.models.load_model('face_mask_cat_Xception.h5')

Cargar el video donde se realizará la inferencia

In [ ]:
cap = cv2.VideoCapture('videoplayback.mp4')

Definir dónde se guardará el video generado

In [ ]:
res = (640, 360)  # resulotion
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec
out = cv2.VideoWriter('video.mp4', fourcc, 20.0, res)

Definición de los colores y textos que se mostrarán en el video

In [ ]:
labels_text = {
    0: 'Incorrecto',
    1: 'Correcto',
    2: 'Sin mascarilla'
}
labels_color = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    2: (255, 0, 0)
}

Leer cada frame del video y realizar la inferencia sobre cada frame, ir guardando el nuevo video generado

In [ ]:
while True:
    success, image = cap.read()
    if not success:
        break
    pixels = asarray(image)
    rects = detector.detect_faces(pixels)
    for (i, rect) in enumerate(rects):
        x1, y1, width, height = rect['box']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        img_cropped = image[y1:y2, x1:x2]
        img_cropped = cv2.resize(img_cropped,
                                 dsize=(150, 150),
                                 interpolation=cv2.INTER_CUBIC)
        img_cropped = img_cropped.reshape(
            (1, img_cropped.shape[0], img_cropped.shape[1], img_cropped.shape[2]))

        preds = model.predict(img_cropped)[0]

        label = labels_text[preds.argmax(axis=-1)]
        color = labels_color[preds.argmax(axis=-1)]

        cv2.rectangle(image,
                      (x1, y1),
                      (x2, y2), color, 2)
        cv2.putText(image, label,
                    (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    out.write(image)


Cerrar el archivo de salida

In [ ]:
out.release()